In [1]:
import numpy as np
import scipy.io as scio
import pandas as pd

import MST_Graph
import VeFC_mst

import VFC_dyad
import Idiff
import Idiff_sit

# 输入数据路径
GROUP1_DATA2021_FOLDER = 'data2021/Class1-Parent'#亲子组
GROUP2_DATA2021_FOLDER = 'data2021/Class2-Stranger'#陌生人组
GROUP1_DATA2023_FOLDER = 'data2023/Class1-Parent'#亲子组
GROUP2_DATA2023_FOLDER = 'data2023/Class2-Stranger'#陌生人组
GROUP1_DATAALL_FOLDER = 'data_all/Class1-Parent'#亲子组
GROUP2_DATAALL_FOLDER = 'data_all/Class2-Stranger'#陌生人组

# 输入数据结构：
# 每个数据文件包含6种拼图场景，其中前面的1/2表示儿童拼或者成人拼，
# 后面的1/2/3表示单独/陪伴/合作,只关注3（合作）
sit_c1 = 'ca11' #儿童拼-单独
sit_a1 = 'ca21' #成人拼-单独
sit_c2 = 'ca12' #儿童拼-陪伴
sit_a2 = 'ca22' #成人拼-陪伴
sit_c3 = 'ca13' #儿童拼-合作
sit_a3 = 'ca23' #成人拼-合作
# 每种情况包含4个cell文件，分别对应4个频段：1-3，3-6，6-9，9-12
delta,theta,alpha,belta=0,1,2,3
freq_all=['delta','theta','alpha','belta']
# 每个cell文件的格式为通道数*时间点数*被试数
# 通道数: 其中1-28为儿童，29-56为成人，通过subc选择类型
# 28个通道及其对应的索引为：
channame_full = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','T7','T8','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_all = list(range(0,28))
# 26个通道及其对应的索引为：
channame_test = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_test = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

# 参数设置
chan_num = 28 #通道数
edge_num = int(chan_num*(chan_num-1)/2) #边数
Nuptran_indices = VFC_dyad.uptran_mask(chan_num) #矩阵上三角列向量的位置
Euptran_indices = VFC_dyad.uptran_mask(edge_num) #矩阵上三角列向量的位置
Vtype='uptran'

# itype指边连接强度的计算方式，'p','c','d'分别代表：PLV复指数、corr相关、delta做差
itype_all=['e','c','d']
itype=itype_all[0]
# corr_type指边与边之间共波动特性的计算方式，'cos','pearson','delta'分别代表：
# 余弦相似度，皮尔逊相关系数，直接做差
corr_type_all=['cos','pearson','delta']
corr_type=corr_type_all[0]

tp=2000

sit=sit_c3 #拼图场景
freq=theta #频段
subc='child'  #对象类型 

## 1、神经活动（包括拼图动作和状态）识别性，总体Idiff

### 计算nFC方法的结果，用以比较

In [2]:
motion_comp = [sit_c1,sit_c2,sit_c3]
I_nFC_all=Idiff_sit.Idiff_motion_nFC_all(GROUP_DATA_FOLDER=GROUP1_DATA2021_FOLDER,motion_comp=motion_comp,
                    freq=freq,subc=subc,corr_type=corr_type)

# 转换为DataFrame格式
I_nFC_allt=I_nFC_all.T
df_nFC = pd.DataFrame(I_nFC_allt)
df_nFC.columns=['I_diff-nFC','I_self-nFC','I_other-nFC','Change_rate-nFC','Diff_rate-nFC'] #设置列名

In [3]:
# 输出数据所要导出到的Excel文件
output_excel='output/Diff_state_mst_all.xlsx'
df=pd.DataFrame() #构造原始数据文件
df.to_excel(output_excel)
df_nFC

,I_diff-nFC,I_self-nFC,I_other-nFC,Change_rate-nFC,Diff_rate-nFC
0,0.26,0.76,0.76,0.34,0.34
1,0.13,0.75,0.75,0.17,0.17
2,0.70,0.73,0.73,0.96,0.96
3,0.12,0.79,0.79,0.15,0.15
4,0.38,0.76,0.75,0.51,0.51
5,0.23,0.78,0.77,0.30,0.30
6,-0.44,0.75,0.76,-0.57,-0.58
7,0.43,0.76,0.75,0.57,0.57
8,-0.11,0.74,0.75,-0.15,-0.15
9,-0.26,0.75,0.75,-0.35,-0.35


In [ ]:
# 创建一个新的工作表，命名为：场景+对象类型+频段
new_sheet_name = 'nFC'+'-'+subc+'-'+freq_all[freq] 
# # 比较各个分支的结果
# new_sheet_name = itype+'-'+corr_type
excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl',if_sheet_exists='replace')
# 保存至excel
df_nFC.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer._save()

### 方法：使用最小生成树MST对VeFC序列矩阵降维

In [6]:
import VeFC_mst
dyad_all = Idiff.file_path_infd(GROUP1_DATAALL_FOLDER)

def Idiff_mst_mn_efc(eFC_mean_sort,mst_num):
    edge_list1,edge_list2=[],[]

    for mn in range(mst_num):
        sort = eFC_mean_sort[:,edge_num+mn*2][0]
        idx_i = sort//edge_num
        idx_j = sort%edge_num
    # 边列表1，2
        edge_list1 = np.append(edge_list1,idx_i)
        edge_list2 = np.append(edge_list2,idx_j)

    edge_list = np.c_[edge_list1,edge_list2]
    VeFC_mst_alld = VeFC_mst.VeFC_mst_all(dyad_all,sit,freq,subc,corr_type,tp,
                             edge_list)
    I_diff,I_self,I_other,I_self_mean,per_change,per_diff=Idiff.cal_Iden(VeFC_mst_alld,trn_mode,'pearsonr')
# print('%.2f'%I_diff,'%.2f'%I_self_mean,'%.2f'%I_other)
# 保留两位小数并记录数据
    I_concat = [I_diff,I_self_mean,I_other,per_change,per_diff]
    I_eFC_mst = np.around(I_concat,2)

    return VeFC_mst_alld,I_eFC_mst


In [7]:
# eFC_mean_all_a3 = VeFC_mst.eFC_all_mean(dyad_all,sit_a3,freq,subc,corr_type,tp)
# eFC_mean_all_c3 = VeFC_mst.eFC_all_mean(dyad_all,sit_c3,freq,subc,corr_type,tp)
# eFC_mean_all = (eFC_mean_all_a3+eFC_mean_all_c3)/2
# # 取绝对值
# eFC_mean_abs = np.abs(eFC_mean_all)
# eFC_mean_1d = eFC_mean_abs.reshape([1,len(eFC_mean_abs)**2])
# eFC_mean_1d_inv = -eFC_mean_1d
# eFC_mean_sort = eFC_mean_1d_inv.argsort()

In [8]:
dim_idx = [edge_num]

I_eFC_mst_all = Idiff_sit.Idiff_motion_eFC_all(GROUP1_DATAALL_FOLDER,motion_comp,freq,subc,corr_type,None)
# 转换为DataFrame格式
I_eFC_mst_allt=I_eFC_mst_all.T
df_eFC1 = pd.DataFrame(I_eFC_mst_allt)
# 添加列名和索引
df_eFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名
df_eFC1

ValueError: operands could not be broadcast together with shapes (378,378) (0,) 

In [ ]:
# 输出数据所要导出到的Excel文件
output_excel='output/Diff_motion_mst_all.xlsx'

# 创建一个新的工作表，命名为：场景+对象类型+频段
new_sheet_name = 'eFC'+'-'+subc+'-'+freq_all[freq] 
# # 比较各个分支的结果
# new_sheet_name = itype+'-'+corr_type
excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl',if_sheet_exists='replace')
# 保存至excel
df_eFC1.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer.save()

C:\Users\吴诚成\AppData\Local\Temp\ipykernel_6624\819090789.py:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()


## 个体识别性-统计检验

#### Bootstrapping
使用Bootstrapping方法进行重抽样：30% = 15组，1000次

In [13]:
import Idiff_bootstrap

I_eFC_all,I_nFC_all = Idiff_bootstrap.Idiff_boot(res_time=1000,yita=0.3,VeFC_mode=VeFC_mst_alld,VnFC_mode=VnFC_mst_alld,
         trn_mode=trn_mode)

In [14]:
# 转换为DataFrame格式
I_eFC_allt=I_eFC_all.T
df_eFC1 = pd.DataFrame(I_eFC_allt)
# 添加列名和索引
df_eFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名

# 转换为DataFrame格式
I_nFC_allt=I_nFC_all.T
df_nFC1 = pd.DataFrame(I_nFC_allt)
# 添加列名和索引
df_nFC1.columns=['I_diff','I_self','I_other','Change_rate','Diff_rate'] #设置列名

In [15]:
output_excel_stats = 'output/Diff_dyad_mst_boot_all.xlsx'
df=pd.DataFrame() #构造原始数据文件
df.to_excel(output_excel_stats)

# 创建一个新的工作表，命名为：场景+对象类型+频段
new_sheet_name = sit+'-'+subc+'-'+freq_all[freq] 
# # 比较各个分支的结果
# new_sheet_name = itype+'-'+corr_type
excel_writer = pd.ExcelWriter(output_excel_stats, mode='a',engine='openpyxl')
'''df = pd.DataFrame({new_sheet_name: []})
df.to_excel(excel_writer, sheet_name=new_sheet_name, index=False)
excel_writer.save()'''
# 亲子组1个组的拼接
df_all = pd.concat([df_eFC1,df_nFC1],axis=1)

# 保存至excel
df_all.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer.save()

C:\Users\吴诚成\AppData\Local\Temp\ipykernel_13160\1800512962.py:18: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
